In [1]:
!pip install sentence-transformers transformers accelerate bitsandbytes trectools

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for trectools: filename=trectools-0.0.50-py3-none-any.whl size=28583 sha256=aeb7f4cce866eb25b73a233da3540e21ad1dfcbf002e6e67cff3dba03eb4c677
  Stored in directory: /root/.cache/pip/wheels/67/23/68/7e98dcdfd7724e9b433a040c4545f11a6b8cfc083678ea9cbc
Successfully built trectools


In [2]:
import json
import random
import torch
import numpy as np
from tqdm import tqdm
from collections import defaultdict

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM


2026-01-03 18:02:03.132302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767463323.365488      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767463323.427942      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767463323.977501      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767463323.977539      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767463323.977542      55 computation_placer.cc:177] computation placer alr

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
torch.backends.cuda.matmul.allow_tf32 = True

In [4]:
DATA_DIR = "/kaggle/input/joker-files"

with open(f"{DATA_DIR}/joker_task1_retrieval_corpus25_EN.json") as f:
    corpus = json.load(f)

with open(f"{DATA_DIR}/joker_task1_retrieval_queries_train25_EN.json") as f:
    queries_train = json.load(f)

with open(f"{DATA_DIR}/joker_task1_retrieval_queries_test25_EN.json") as f:
    queries_test = json.load(f)

with open(f"{DATA_DIR}/joker_task1_retrieval_qrels_train25_EN.json") as f:
    qrels_train = json.load(f)

print(corpus[0], queries_train[0], qrels_train[0])

{'docid': '1', 'text': 'He has a green body, no visible nose, and lives in a trash can.'} {'qid': '8', 'query': 'colors'} {'qid': 8, 'docid': 151, 'qrel': 1}


In [5]:
retriever = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device=DEVICE
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
doc_texts = [d["text"] for d in corpus]
doc_ids = [d["docid"] for d in corpus]

doc_embeddings = retriever.encode(
    doc_texts,
    convert_to_numpy=True,
    normalize_embeddings=True,
    batch_size=256,
    show_progress_bar=True
)

Batches:   0%|          | 0/304 [00:00<?, ?it/s]

In [7]:
clf_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(clf_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    clf_name,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

model.config.pad_token_id = tokenizer.pad_token_id
model.eval()

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((4096,)

In [8]:
def build_prompt(query, doc):
    return f"""[INST]
Query:
{query}

Document:
{doc}

Question:
Is this document humorous and relevant to the query?

Answer yes or no.
[/INST]
"""

In [9]:
YES_TOKEN = tokenizer.encode(" yes", add_special_tokens=False)[0]
NO_TOKEN = tokenizer.encode(" no", add_special_tokens=False)[0]

In [10]:
@torch.no_grad()
def humor_scores_qd(queries, docs, batch_size=8):
    scores = []

    for i in tqdm(range(0, len(docs), batch_size)):
        batch_q = queries[i:i+batch_size]
        batch_d = docs[i:i+batch_size]

        prompts = [
            build_prompt(q, d if isinstance(d, str) else "")
            for q, d in zip(batch_q, batch_d)
        ]

        enc = tokenizer(
            prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=2048
        ).to(DEVICE)

        outputs = model(**enc)
        logits = outputs.logits[:, -1, :]

        yes_logits = logits[:, YES_TOKEN]
        no_logits = logits[:, NO_TOKEN]

        probs = torch.softmax(
            torch.stack([no_logits, yes_logits], dim=1),
            dim=1
        )[:, 1]

        scores.extend(probs.cpu().numpy())

    return np.array(scores)

In [11]:
SIM_THRESHOLD = 0.0

TOP_K_SEMANTIC = 300

def retrieve(query, top_k=120):
    q_emb = retriever.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0]

    sim_scores = np.dot(doc_embeddings, q_emb)

    sem_idx = np.argsort(sim_scores)[::-1][:TOP_K_SEMANTIC]

    cand_docs = [doc_texts[i] for i in sem_idx]
    cand_queries = [query] * len(cand_docs)

    humor_scores = humor_scores_qd(
        cand_queries,
        cand_docs,
        batch_size=8
    )

    order = np.argsort(humor_scores)[::-1][:top_k]

    return [
        {
            "docid": doc_ids[sem_idx[i]],
            "rank": r + 1,
            "score": float(humor_scores[i])
        }
        for r, i in enumerate(order)
    ]

In [12]:
train_predictions = []

for q in tqdm(queries_train):
    res = retrieve(q["query"], top_k=120)
    for r in res:
        train_predictions.append({
            "qid": q["qid"],
            "docid": r["docid"],
            "rank": r["rank"],
            "score": r["score"]
        })

100%|██████████| 12/12 [04:37<00:00, 23.13s/it]


In [13]:
with open("run_train.txt", "w") as f:
    for r in train_predictions:
        f.write(f"{r['qid']} Q0 {r['docid']} {r['rank']} {r['score']} run\n")

with open("qrels_train.txt","w") as f:
    for q in qrels_train:
        f.write(f"{q['qid']} 0 {q['docid']} {q['qrel']}\n")

In [14]:
from trectools import TrecRun, TrecQrel, TrecEval

run = TrecRun("run_train.txt")
qrels = TrecQrel("qrels_train.txt")

ev = TrecEval(run, qrels)

metrics = {
    "map": ev.get_map(),
    "recip_rank": ev.get_reciprocal_rank(),
    "ndcg_5": ev.get_ndcg(5),
    "ndcg_10": ev.get_ndcg(10),
    "ndcg_20": ev.get_ndcg(20),
    "P_5": ev.get_precision(5),
    "P_10": ev.get_precision(10),
    "P_20": ev.get_precision(20),
    "recall_5": ev.get_recall(5),
    "recall_10": ev.get_recall(10),
    "recall_20": ev.get_recall(20),
}

metrics

/usr/local/lib/python3.12/dist-packages/trectools/trec_eval.py:311: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  selection = selection[~selection["rel"].isnull()].groupby("query").first().copy()


{'map': np.float64(0.10450990036803987),
 'recip_rank': np.float64(0.3493091703618019),
 'ndcg_5': np.float64(0.2948569331398149),
 'ndcg_10': np.float64(0.29584069594740486),
 'ndcg_20': np.float64(0.29983411604921434),
 'P_5': np.float64(0.20000000000000004),
 'P_10': np.float64(0.25),
 'P_20': np.float64(0.2416666666666667),
 'recall_5': np.float64(0.029925078861249076),
 'recall_10': np.float64(0.072558194366705),
 'recall_20': np.float64(0.16947600272068356)}

In [15]:
test_predictions = []

for q in tqdm(queries_test):
    res = retrieve(q["query"], top_k=120)
    for r in res:
        test_predictions.append({
            "qid": q["qid"],
            "docid": r["docid"],
            "rank": r["rank"],
            "score": r["score"]
        })

with open("run_test.txt", "w") as f:
    for r in test_predictions:
        f.write(f"{r['qid']} Q0 {r['docid']} {r['rank']} {r['score']} run\n")

100%|██████████| 219/219 [1:25:09<00:00, 23.33s/it]


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/joker_files/joker_task1_retrieval_qrels_test25_EN.json'